In [1]:
# VesterAI - Notebook 08: Streamlit Chatbot App

"""
Goal:
Create a chatbot interface using Streamlit that:
- Accepts natural language questions
- Uses fine-tuned LLaMA 2 to respond with investor-ready insights
- Pulls in current or selected date’s data from your dataset
"""

'\nGoal:\nCreate a chatbot interface using Streamlit that:\n- Accepts natural language questions\n- Uses fine-tuned LLaMA 2 to respond with investor-ready insights\n- Pulls in current or selected date’s data from your dataset\n'

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# Step 1: Load PEFT config
peft_model_path = "../models/llama2_finetuned"
peft_config = PeftConfig.from_pretrained(peft_model_path)

# Step 2: Load base model (same one used during fine-tuning)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit=True
)

# Step 3: Load LoRA adapter weights
model = PeftModel.from_pretrained(base_model, peft_model_path)

# Step 4: Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/tandel.r/.local/lib/python3.8/site-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [8]:
import pandas as pd

df = pd.read_csv("../data/processed/AAPL_model_data.csv", parse_dates=["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Use most recent data row
latest = df.iloc[-1]

In [9]:
def build_prompt(user_question, row):
    context = f"""Date: {row['Date'].strftime('%Y-%m-%d')}
Stock Close: {row['Close']:.2f}
Return: {row['return']:.4f}
Twitter Sentiment: {row['twitter_sentiment']:.2f}
News Sentiment: {row['news_sentiment']:.2f}
Reddit Sentiment: {row.get('reddit_sentiment', 0):.2f}
RSI: {row['rsi_14']:.2f}
MACD: {row['macd']:.4f}
OBV: {row['obv']:.2f}"""

    return f"""### Instruction:
{user_question}

### Input:
{context}

### Response:
"""

In [10]:
def get_llm_response(question, row):
    prompt = build_prompt(question, row)
    output = llm(prompt, max_new_tokens=200)[0]["generated_text"]
    return output.replace(prompt, "").strip()

In [12]:
!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 8.6 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 207 kB 119.9 MB/s eta 0:00:01
     |████████████████████████████████| 658 kB 47.6 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 331 kB/s s eta 0:00:01
     |████████████████████████████████| 6.9 MB 84.4 MB/s eta 0:00:01
     |████████████████████████████████| 320 kB 88.6 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 423 kB/s  eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
spyder 4.2.5 requires watchdog<2.0.0,>=0.10.3, but you have watchdog 4.0.2 which is incompatible.


In [13]:
import streamlit as st

st.title("VesterAI - Financial Sentiment Chatbot")

user_input = st.text_input("Ask a question about today's market sentiment or outlook:")

if st.button("Generate Insight"):
    if user_input:
        response = get_llm_response(user_input, latest)
        st.write("**Response:**")
        st.markdown(response)
    else:
        st.warning("Please enter a question.")

2025-03-25 22:30:47.801 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 22:30:47.874 
  command:

    streamlit run /home/tandel.r/.local/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-25 22:30:47.875 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 22:30:47.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 22:30:47.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 22:30:47.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 22:30:47.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 22:30:47.877 Session state do

Usage: streamlit [OPTIONS] COMMAND [ARGS]...
Try 'streamlit --help' for help.

Error: No such command 'run/.scratch/tandel.r/VesterAI/streamlit_app/app.py'.


In [26]:
!python -m venv vesterai-env
!vesterai-env\\Scripts\\activate    

/bin/bash: vesterai-env\Scripts\activate: command not found


In [27]:
!pip install streamlit transformers peft bitsandbytes accelerate pandas

Defaulting to user installation because normal site-packages is not writeable


In [1]:
%cd ..

/scratch/tandel.r/VesterAI


In [2]:
!/scratch/tandel.r/VesterAI

/bin/bash: /scratch/tandel.r/VesterAI: Is a directory


In [ ]:
%cd streamlit_app
!streamlit run app.py

/scratch/tandel.r/VesterAI/streamlit_app
